In [ ]:
# install dependencies
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install lxml

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [71]:
filepath = 'data/scouting-data.html'

dfs = pd.read_html(filepath, encoding='utf8')
dfs[0].head()

,Rec,Inf,Name,Age,Position,Club,Division,Mins,Gls,xG,...,Int/90,Clr/90,Hdr %,K Hdrs/90,Hdrs W/90,Pres A/90,Pres C/90,Sprints/90,Dist/90,Distance
0,- - E -,Yth,Adrián Samper,18,"D (C), DM, M (C)",Sevilla,Spanish First Division,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,- - -,Inj,Thodoros Mouratidis,17,D (C),Fulham,Sky Bet Championship,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,- - -,NaN,Pascal Adeyemi,17,AM (L),Fulham,Sky Bet Championship,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,- - -,Yth,Tom Hardy,17,M/AM (L),Nottm Forest,English Premier Division,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,- - -,Yth,Callum Wilson,17,ST (C),Leeds,English Premier Division,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [79]:
print("Number of players:", dfs[0].shape[0])
print("Number of fields:", dfs[0].shape[1])

Number of players: 13188
Number of fields: 42


In [80]:
players.columns

Index(['Name', 'Age', 'Position', 'Club', 'Division', 'Mins', 'Gls', 'xG',
       'xG/90', 'NP-xG/90', 'Shot %', 'Shot/90', 'Conv %', 'Ast', 'xA',
       'xA/90', 'Pas %', 'Pr passes/90', 'Ps C/90', 'OP-KP/90', 'OP-Cr %',
       'OP-Crs C/90', 'Ch C/90', 'Drb/90', 'Poss Won/90', 'Poss Lost/90',
       'Tck R', 'K Tck/90', 'Tck/90', 'Blk/90', 'Int/90', 'Clr/90', 'Hdr %',
       'K Hdrs/90', 'Hdrs W/90', 'Pres A/90', 'Pres C/90', 'Sprints/90',
       'Dist/90 (mi)'],
      dtype='object')

In [81]:
# data processing

players = dfs[0].copy()

percent_cols = ['Shot %', 'Conv %', 'Pas %', 'OP-Cr %', 'Tck R', 'Hdr %']
nonnumeric_cols = ['Name', 'Age', 'Position', 'Club', 'Division']
int_cols = ['Mins', 'Gls', 'Ast']

players = players.replace('-', 0)
# 'Rec', 'Info' are not useful. For some reasons 'Dist/90' are all 0.0s in the data
players = players.drop(['Rec', 'Inf', 'Dist/90'], axis=1)

# convert % string to float
players[percent_cols] = players[percent_cols].apply(lambda x: x.str.rstrip('%').astype(float) / 100)
# get rid of "mi"
players['Distance'] = players['Distance'].str[:-2]

# convert to correct types
for col in players.columns:
    if col not in nonnumeric_cols and col not in int_cols:
        players[col] = players[col].astype(float)
    if col in int_cols:
        players[col] = players[col].astype(int)

# exclude players with 0 minutes
players = players[players['Mins'] > 0]
# manually calculate and add Dist/90 back to dataframe 
players['Dist/90 (mi)'] = round(players['Distance'] / players['Mins'] * 90, 2)
players = players.drop('Distance', axis=1)

# positional groups: gk, centerbacks, fullbacks, midfielders, attacking mid/wingers, forwards
# players['Postion Group'] = 

players.head()

,Name,Age,Position,Club,Division,Mins,Gls,xG,xG/90,NP-xG/90,...,Blk/90,Int/90,Clr/90,Hdr %,K Hdrs/90,Hdrs W/90,Pres A/90,Pres C/90,Sprints/90,Dist/90 (mi)
19,Nicolas Jackson,30,"AM (RLC), ST (C)",Chelsea,English Premier Division,74,0,0.00,0.00,0.00,...,0.0,3.65,0.0,0.00,0.0,0.0,8.51,2.43,7.3,9.36
21,Dorgéles Néné,28,"AM (RLC), ST (C)",Mainz 05,Bundesliga,20,0,0.00,0.00,0.00,...,0.0,0.00,0.0,NaN,0.0,0.0,0.00,0.00,0.0,7.65
34,Moussa Kanté,27,"M (RC), AM (RLC)",Red Star FC,Ligue 2 BKT,75,0,0.14,0.17,0.17,...,0.0,1.20,1.2,0.00,0.0,0.0,13.20,1.20,14.4,9.36
49,Pape Habib Guèye,32,"AM (C), ST (C)",Willem II,Keuken Kampioen Divisie,46,1,0.18,0.00,0.00,...,0.0,0.00,0.0,0.17,0.0,0.0,0.00,0.00,0.0,5.67
54,Montassar Talbi,33,D (C),Nottm Forest,English Premier Division,29,0,0.00,0.00,0.00,...,0.0,0.00,0.0,0.50,0.0,0.0,0.00,0.00,0.0,9.93


In [60]:
# players['Mins'].describe()
# players['Dist/90'].describe()
# players['Distance'].describe()

count    6903.000000
mean       43.834043
std        35.258350
min         0.000000
25%        10.400000
50%        37.700000
75%        73.300000
max       149.300000
Name: Distance, dtype: float64

In [76]:
players.head()

,Name,Age,Position,Club,Division,Mins,Gls,xG,xG/90,NP-xG/90,...,Blk/90,Int/90,Clr/90,Hdr %,K Hdrs/90,Hdrs W/90,Pres A/90,Pres C/90,Sprints/90,Dist/90 (mi)
19,Nicolas Jackson,30,"AM (RLC), ST (C)",Chelsea,English Premier Division,74,0,0.00,0.00,0.00,...,0.0,3.65,0.0,0.00,0.0,0.0,8.51,2.43,7.3,9.36
21,Dorgéles Néné,28,"AM (RLC), ST (C)",Mainz 05,Bundesliga,20,0,0.00,0.00,0.00,...,0.0,0.00,0.0,NaN,0.0,0.0,0.00,0.00,0.0,7.65
34,Moussa Kanté,27,"M (RC), AM (RLC)",Red Star FC,Ligue 2 BKT,75,0,0.14,0.17,0.17,...,0.0,1.20,1.2,0.00,0.0,0.0,13.20,1.20,14.4,9.36
49,Pape Habib Guèye,32,"AM (C), ST (C)",Willem II,Keuken Kampioen Divisie,46,1,0.18,0.00,0.00,...,0.0,0.00,0.0,0.17,0.0,0.0,0.00,0.00,0.0,5.67
54,Montassar Talbi,33,D (C),Nottm Forest,English Premier Division,29,0,0.00,0.00,0.00,...,0.0,0.00,0.0,0.50,0.0,0.0,0.00,0.00,0.0,9.93


In [77]:
players['xG'].describe()

count    6903.000000
mean        0.731919
std         1.088646
min         0.000000
25%         0.000000
50%         0.280000
75%         0.980000
max        11.560000
Name: xG, dtype: float64

In [78]:
players['Gls'].describe()

count    6903.000000
mean        0.699551
std         1.343975
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max        15.000000
Name: Gls, dtype: float64

In [89]:
players.groupby('Position').size().sort_values()

Position
WB/M/AM (R), ST (C)            1
D (C), DM, M (C), AM (R)       1
D (C), DM, M (C), AM (RC)      1
D (C), DM, M (L), AM (C)       1
D (C), DM, M (LC), AM (C)      1
                            ... 
DM, M/AM (C)                 259
ST (C)                       300
D (C)                        374
DM, M (C)                    431
GK                           518
Length: 468, dtype: int64